<a href="https://colab.research.google.com/github/theharshithh/onfinace-assignment/blob/main/connectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tldr 07
1. use cohere's connectors and induced.ai api + yfinance to feed the raw updated data -> https://docs.cohere.com/docs/connectors?_gl=1*1h90opw*_gcl_au*MTM5OTE1Mzc2NC4xNzEyOTQ0NTU1*_ga*NzgyODYyMzI3LjE3MTI5NDQ1OTk.*_ga_CRGS116RZS*MTcxMzAyNjc0NS4zLjEuMTcxMzAyNzIwMy42MC4wLjA.
2. trying for an alternative of fine-tuning

suggestions
2. use LLaVA or CLIP to decode images n parse it
1. better table conversations ref : https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb

and https://github.com/langchain-ai/langchain/blob/master/cookbook/Multi_modal_RAG.ipynb


In [3]:
!pip --quiet install langchain openai tiktoken chromadb langchain_openai

In [9]:
!pip --quiet install langchain_cohere cohere pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.0 MB/s eta 0:00:00


In [32]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
# from langchain_cohere import CohereEmbeddings
import cohere
from google.colab import userdata
cohere_api_key = userdata.get('COHERE_API_KEY')
openai_api_key = userdata.get('OPENAI_API_KEY')
from langchain.prompts.example_selector import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)

co = cohere.Client(cohere_api_key)
from langchain_community.document_transformers import (
    LongContextReorder,
)


In [10]:
# def read_doc(dir):
#     file_loader = PyPDFDirectoryLoader(dir)
#     docs = file_loader.load()
#     return docs

file_loader = PyPDFLoader('dataset/sample_data.pdf')
docs = file_loader.load()
docs

[Document(page_content=' \nJinesh Gandhi  – Research Analyst  (Jinesh@MotilalOswal.com)  \nResearch Analyst : Amber  Shukla  (Amber.Shukla@ Motilal Oswal.com ) | Aniket Desai (Aniket .Desai @MotilalOswal.com)  \n \nInvestors are advised to refer through important disclosures made at the last page of the Research Report.  \nMotilal Oswal research is available on www.motilaloswal.com/Institutional -Equities, Bloomberg, Thomson Reuters, Factset and S&P Capital.  \n @ \n \n \nMHCV likely to outperform despite multiple challenges  \nInitial signs of demand moderati on in PV; expect 2W volumes to grow YoY  \n\uf06e Our interaction with leading channel partners indicate s some moderation in \ndemand  due to erratic rainfall during the month , especially in key Northern and \ncentral regions and lower enquiries  during the ongoing festive season.  Despite \nthis, retails for both MHCV and 2Ws are expected to grow while PV volume \ngrowth is expected to  remain flattish.  We note  that most of 

In [12]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

output1 = '''
Based on the data provided for 2-wheeler segments from various manufacturers for the specified period:

1. **Hero MotoCorp** shows positive year-on-year growth with units sold increasing from 49,44,148 to 52,43,369 (a **6.1%** increase). This indicates resilient demand, positioning Hero MotoCorp strongly in terms of sales volume in the 2-wheeler segment.

2. **TVS Motor** similarly demonstrated substantial growth. They showed an especially strong performance in scooters, with numbers increasing from 9,21,939 to 12,97,275, exhibiting an impressive **40.7%** year-on-year growth. Motorcycle sales remained almost flat while mopeds decreased by **-5.0%**. However, due to the strong scooter numbers, TVS Motor allows for considerable brand perception and growth.

3. **Bajaj Auto**, on the other hand, experienced a decrease in 2-wheeler sales, dropping from 43,08,433 to 39,16,077, marking a drop of **-9.1%** year-on-year which indicates a relatively weaker market position in comparison.

In the context mentioned and focusing purely on recent market performance and growth in numbers, **TVS Motor** especially shines in the scooter segment, which highlights them as a potential segment winner, closely followed by Hero MotoCorp due to its large volume and overall positive growth in this document period. Bajaj Auto appears to struggle compared to the other two in this specific period evaluated.
'''

output2= '''
Based on the data provided, the two-wheeler segment winner can be determined by analyzing the sales and growth figures of the key players within the segment. In this case, we can see that Hero MotoCorp and TVS Motor are the two significant players to analyze.

Considering the data:
1. Hero MotoCorp's sales figures show a growth rate in combined two-wheeler sales by 6.1% YoY, totaling 52,43,369. Despite a slight decline in the specific year, it overall maintains solid numbers.

2. TVS Motor's sales figures reflect a slightly lower growth rate compared to Hero MotoCorp, at 10.5% YoY with the total sales of 36,57,093. However, TVS Motor does show positive growth in both motorcycle and scooter segments.

On comparative analysis:
- Hero MotoCorp maintains higher overall sales in the two-wheeler segment.
- However, in terms of growth rate, TVS Motor outperforms Hero MotoCorp with a 10.5% growth compared to Hero MotoCorp's 6.1% increase YoY.
'''
examples = [
    {"input": "Considering 2 wheeler demand, who seems to be the segment winner?", "output": output1},
    {"input": "Bassed on the report, who is leading the two wheeler space right now?", "output": output2}
    # {"input": "energetic", "output": "lethargic"},
    # {"input": "sunny", "output": "gloomy"},
    # {"input": "windy", "output": "calm"},
]

creating chunks for the `primary_data`

In [13]:
primary_data = docs
primary_data
# primary_data = [doc.page_content for doc in docs]

[Document(page_content=' \nJinesh Gandhi  – Research Analyst  (Jinesh@MotilalOswal.com)  \nResearch Analyst : Amber  Shukla  (Amber.Shukla@ Motilal Oswal.com ) | Aniket Desai (Aniket .Desai @MotilalOswal.com)  \n \nInvestors are advised to refer through important disclosures made at the last page of the Research Report.  \nMotilal Oswal research is available on www.motilaloswal.com/Institutional -Equities, Bloomberg, Thomson Reuters, Factset and S&P Capital.  \n @ \n \n \nMHCV likely to outperform despite multiple challenges  \nInitial signs of demand moderati on in PV; expect 2W volumes to grow YoY  \n\uf06e Our interaction with leading channel partners indicate s some moderation in \ndemand  due to erratic rainfall during the month , especially in key Northern and \ncentral regions and lower enquiries  during the ongoing festive season.  Despite \nthis, retails for both MHCV and 2Ws are expected to grow while PV volume \ngrowth is expected to  remain flattish.  We note  that most of 

## create the embeddings using hf ede by baai

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
spilit_texts = text_splitter.split_documents(primary_data)
spilit_texts

[Document(page_content='Jinesh Gandhi  – Research Analyst  (Jinesh@MotilalOswal.com)  \nResearch Analyst : Amber  Shukla  (Amber.Shukla@ Motilal Oswal.com ) | Aniket Desai (Aniket .Desai @MotilalOswal.com)  \n \nInvestors are advised to refer through important disclosures made at the last page of the Research Report.  \nMotilal Oswal research is available on www.motilaloswal.com/Institutional -Equities, Bloomberg, Thomson Reuters, Factset and S&P Capital.  \n @ \n \n \nMHCV likely to outperform despite multiple challenges  \nInitial signs of demand moderati on in PV; expect 2W volumes to grow YoY  \n\uf06e Our interaction with leading channel partners indicate s some moderation in \ndemand  due to erratic rainfall during the month , especially in key Northern and \ncentral regions and lower enquiries  during the ongoing festive season.  Despite \nthis, retails for both MHCV and 2Ws are expected to grow while PV volume \ngrowth is expected to  remain flattish.  We note  that most of the

using a sep vectodb for storing trash texts from the report

In [15]:
persist_directory = 'db-trash'

#openai embeddings -> converts texts to vecs
embedding = OpenAIEmbeddings(openai_api_key = openai_api_key)
# embedding = OpenAIEmbeddings(openai_api_key = openai_api_key, model="text-embedding-3-small")


#load the spilt chunk vectors into the vectordb
vectordb = Chroma.from_documents(documents=spilit_texts, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()
vectordb = None
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [16]:

# Use your trash data retriever to get the relevant documents
trash_retriever = vectordb.as_retriever(search_kwargs={"k":20}) #k: 20 for more context window to llm
trash_text = trash_retriever.get_relevant_documents('Get me the SEBI disclosure and the unnecessary disclaimers and terms and conditions')

trash_metadata = [doc.metadata for doc in trash_text]
main_data_filtered = [doc for doc in primary_data if doc.metadata not in trash_metadata]
main_data_filtered


# trash_text_content = [doc.page_content for doc in trash_text]
# cleaned_chunk = primary_data
# for trash in trash_text_content:
#     cleaned_chunk = cleaned_chunk.replace(trash, "")


[Document(page_content=' \nJinesh Gandhi  – Research Analyst  (Jinesh@MotilalOswal.com)  \nResearch Analyst : Amber  Shukla  (Amber.Shukla@ Motilal Oswal.com ) | Aniket Desai (Aniket .Desai @MotilalOswal.com)  \n \nInvestors are advised to refer through important disclosures made at the last page of the Research Report.  \nMotilal Oswal research is available on www.motilaloswal.com/Institutional -Equities, Bloomberg, Thomson Reuters, Factset and S&P Capital.  \n @ \n \n \nMHCV likely to outperform despite multiple challenges  \nInitial signs of demand moderati on in PV; expect 2W volumes to grow YoY  \n\uf06e Our interaction with leading channel partners indicate s some moderation in \ndemand  due to erratic rainfall during the month , especially in key Northern and \ncentral regions and lower enquiries  during the ongoing festive season.  Despite \nthis, retails for both MHCV and 2Ws are expected to grow while PV volume \ngrowth is expected to  remain flattish.  We note  that most of 

## spiliting the filtered texts by `main_data_filtered`

In [17]:
spilit_texts = text_splitter.split_documents(main_data_filtered)

The idea is that when we have a lot of data reports, majority of them are disclosures warnings, so run an filteration and then rag

In [18]:
persist_directory = 'db-filtered'

#openai embeddings -> converts texts to vecs
embedding = OpenAIEmbeddings(openai_api_key = openai_api_key)
# embedding = OpenAIEmbeddings(openai_api_key = openai_api_key, model="text-embedding-3-small")


#load the spilt chunk vectors into the vectordb
vectordb = Chroma.from_documents(documents=spilit_texts, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()
vectordb = None
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [19]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,
    embedding,
    vectordb,
    k=2,
)

In [21]:
user_query = 'Considering the demand for the two wheelers, who is leading the segment winner?'

In [33]:
retriever = vectordb.as_retriever(search_kwargs={"k":8}) #k: 20 for more context window to llm
docs = retriever.get_relevant_documents(user_query)

#reordering the docs
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)
reordered_docs

[Document(page_content='Motorcycles  1,37,970  1,60,522  -14.0  1,26,243  9.3 17,29,976  17,31,459  -0.1 \nScooters  1,05,785  94,747  11.6  1,04,825  0.9 12,97,275  9,21,939  40.7  \nMopeds  38,352  37,649  1.9 35,958  6.7 4,60,251  4,84,305  -5.0 \nThree -Wheelers  10,070  15,036  -33.0  9,124  10.4  1,69,591  1,71,875  -1.3 \nDomestic  2,35,861  1,98,230  19.0  2,22,745  5.9 26,07,757  20,56,387  26.8  \nExports  56,315  1,09,724  -48.7  53,405  5.4 10,49,335  12,53,191  -16.3  \nEicher Motors          Royal Enfield  74,155  67,670  9.6 71,544  3.6 8,36,815  6,02,268  38.9  \nVECV  9,125  8,793  3.8 7,289  25.2  76,821  57,067  34.6  \nEscorts Kubota  10,195  10,074  1.2 7,811  30.5  1,03,180  94,228  9.5', metadata={'page': 2, 'source': 'dataset/sample_data.pdf'}),
 Document(page_content='5% YoY , largely due to  the impact of  unseasonal rains and low yield s for key \ncrops  such as oni on. In Mar’23, overall wholesale volumes for \nPV/CV/tractor/3Ws are estimated to grow by 1%/ 

## LLM chain and rerank3 by cohere


In [24]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere

compressor = CohereRerank(cohere_api_key = cohere_api_key, model = 'rerank-english-v3.0') #   top_n = 3 -> can limit the sources
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever,
)

compressed_docs = compression_retriever.get_relevant_documents(user_query)
compressed_docs
# results = co.rerank(query=user_query, documents=docs, top_n=10, model='rerank-english-v3.0')

checking the difference bw compressor retrival and normal retrival

In [26]:
qa_chain_compressed = RetrievalQA.from_chain_type(llm=ChatOpenAI(openai_api_key=openai_api_key,model="gpt-3.5-turbo", temperature=1.2 ), chain_type="stuff", retriever=compression_retriever, return_source_documents=True)

In [27]:
qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(openai_api_key=openai_api_key,model="gpt-3.5-turbo", temperature=1.2 ), chain_type="stuff", retriever=retriever, return_source_documents=True)

In [28]:
def process_llm_response(llm_response):
    print("Decision:")
    print(llm_response['result'])

    print("\n Source Documents:")
    for source in llm_response["source_documents"]:
        print("- " + source.metadata['source'])

    print("\nRelevant Data:")
    for source in llm_response["source_documents"]:
        print(source.page_content)

In [31]:
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Decision:",
    suffix="Input: {user_query}\nOutput:",
    input_variables=["user_query"],
)
mmr_prompt.format(user_query=user_query)


"Decision:\n\nInput: Considering 2 wheeler demand, who seems to be the segment winner?\nOutput: \nBased on the data provided for 2-wheeler segments from various manufacturers for the specified period:\n\n1. **Hero MotoCorp** shows positive year-on-year growth with units sold increasing from 49,44,148 to 52,43,369 (a **6.1%** increase). This indicates resilient demand, positioning Hero MotoCorp strongly in terms of sales volume in the 2-wheeler segment.\n\n2. **TVS Motor** similarly demonstrated substantial growth. They showed an especially strong performance in scooters, with numbers increasing from 9,21,939 to 12,97,275, exhibiting an impressive **40.7%** year-on-year growth. Motorcycle sales remained almost flat while mopeds decreased by **-5.0%**. However, due to the strong scooter numbers, TVS Motor allows for considerable brand perception and growth.\n\n3. **Bajaj Auto**, on the other hand, experienced a decrease in 2-wheeler sales, dropping from 43,08,433 to 39,16,077, marking a 

compressed qa chain for efficent retrival

In [ ]:
behaviour = '''Think like an expert finance consultant from a top tier firm. Do all the research and analysis for this document.'''
prompt_template = 'Keep the answer crisp. Give me as much numbers as you can. Every Statement of yours should be backed with data'
stop_prompt = f'Dont hallucinate, if u dont know the answer, tell you dont know'
query = f"Here is the user query : {user_query}. Remember this: {behaviour}. Give me the answers like this: {prompt_template}. Keep this in mind - {stop_prompt}"
llm_response = qa_chain_compressed(query)
process_llm_response(llm_response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Decision:
Based on the data provided, the two-wheeler segment winner can be determined by analyzing the sales and growth figures of the key players within the segment. In this case, we can see that Hero MotoCorp and TVS Motor are the two significant players to analyze.

Considering the data:
1. Hero MotoCorp's sales figures show a growth rate in combined two-wheeler sales by 6.1% YoY, totaling 52,43,369. Despite a slight decline in the specific year, it overall maintains solid numbers.

2. TVS Motor's sales figures reflect a slightly lower growth rate compared to Hero MotoCorp, at 10.5% YoY with the total sales of 36,57,093. However, TVS Motor does show positive growth in both motorcycle and scooter segments.

On comparative analysis:
- Hero MotoCorp maintains higher overall sales in the two-wheeler segment.
- However, in terms of growth rate, TVS Motor outperforms Hero MotoCorp with a 10.5% growth compared to Hero MotoCorp's 6.1% increase YoY.

From an expert finance consultant perspe

best version -> heavy credits burning -> deep pockets required

In [ ]:
llm_response = qa_chain(query)
process_llm_response(llm_response)

Decision:
Based on the data provided, the leader in the two-wheeler segment winner can be determined by considering various indicators such as sales volume, growth rates, market share, and customer interest.

1. **Hero MotoCorp**: Known for its wide range of offerings and a substantial market presence, Hero MotoCorp recorded sales of 4,34,165 units in the last period, showcasing consistent growth. Their yearly growth rate of 6.1% is notable compared to other players in the segment.

2. **TVS Motor**: While reporting a marginally lower sales volume compared to Hero MotoCorp at 2,92,177 units, TVS Motor achieved a growth rate of 10.5% in the same period, indicating a stronger growth trajectory.

3. **Bajaj Auto**: Despite experiencing a decline in sales volume to 2,81,922 units, Bajaj Auto maintained its market position and showcased a mix of challenges and opportunities with a yearly growth rate of -9.1%.

Based on the provided data, Hero MotoCorp appears to have a stronger foothold in 

In [ ]:
qa_chain